# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import pprint
import datetime
# import pytemperature
# not a standard library: pip install pytemperature

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
# not a standard library: pip install citipy
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# now
now = datetime.datetime.now()

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

596

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# API used: https://openweathermap.org/api

# by coordinates: api.openweathermap.org/data/2.5/weather?lat=35&lon=139
# by city name: api.openweathermap.org/data/2.5/weather?q=London

# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

# variables
City = []
Cloudiness = []
Country = []
Date = []
Humidity = []
Lat = []
Lng = []
Max_Temp = []
Wind_Speed = []

count = 1

# query_url = url + "appid=" + api_key + "&q=" + city #to do
generic_weather_url = url + "appid=" + api_key +'&units=imperial' + "&q="
pp = pprint.PrettyPrinter(indent=4)

# functionality
print(f'\033[4m' 'Beginning Data Retrieval' '\033[0m') # status update while processing
for count, city in enumerate(cities, start=1):
#     Build query URL 
    query_url = generic_weather_url + city
#     Get weather data
    print(f'Processing Record {count} of Set 1 | {city}')
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    if weather_json['cod'] != 200:
        print(f'City not found. Skipping...')
    else: #include in our dataset
#         pp.pprint(weather_json)
        humidity = float(weather_json['main']['humidity'])
        if humidity <= 100:  #humidity can not be above 100 percent
            City.append(weather_json['name'])
            Cloudiness.append(weather_json['clouds']['all'])
            Country.append(weather_json['sys']['country'])
            Date.append(weather_json['dt'])
            Humidity.append(weather_json['main']['humidity'])
            Lat.append(weather_json['coord']['lat'])
            Lng.append(weather_json['coord']['lon'])
#             Max_Temp.append(pytemperature.k2f(weather_json['main']['temp_max']))
            Max_Temp.append(weather_json['main']['temp_max'])
            Wind_Speed.append(weather_json['wind']['speed'])     

Beginning Data Retrieval
Processing Record 1 of Set 1 | provideniya
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | hilo
Processing Record 4 of Set 1 | carutapera
Processing Record 5 of Set 1 | port alfred
Processing Record 6 of Set 1 | jamestown
Processing Record 7 of Set 1 | bredasdorp
Processing Record 8 of Set 1 | san isidro
Processing Record 9 of Set 1 | barrow
Processing Record 10 of Set 1 | quartz hill
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | ilulissat
Processing Record 13 of Set 1 | chuy
Processing Record 14 of Set 1 | vaini
Processing Record 15 of Set 1 | kawalu
Processing Record 16 of Set 1 | dillon
Processing Record 17 of Set 1 | louisbourg
City not found. Skipping...
Processing Record 18 of Set 1 | tawang
Processing Record 19 of Set 1 | kapaa
Processing Record 20 of Set 1 | butaritari
Processing Record 21 of Set 1 | noumea
Processing Record 22 of Set 1 | rikitea
Processing Record 23 of Set 1 | nipawin
Processing Reco

In [ ]:
# build the dataframe
city_weather_dict = {
    'City':          City,
    'Cloudiness':    Cloudiness,
    'Country':       Country,
    'Date':          Date,
    'Humidity':      Humidity,
    'Lat':           Lat,
    'Lng':           Lng,
    'Max Temp':      Max_Temp,
    'Wind Speed':    Wind_Speed    
}
city_weather_data = pd.DataFrame(city_weather_dict)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_weather_data.to_csv('WeatherPy.csv',index=False)
city_weather_data.info()

In [ ]:
city_weather_data.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
def generic_plot(ax_x,ax_y):
#     Build a scatter plot 
    plt.scatter(city_weather_data[ax_x], city_weather_data[ax_y], marker="o",edgecolors='black')
    x_label='Latitude'
    if ax_y == 'Max Temp':
        y_label = 'Max Temperature (F)'
        plot_title = 'City Latitude vs. Max Temperature'
    elif ax_y == 'Humidity':
        y_label= 'Humidity (%)'
        plot_title = 'City Latitude vs. Humidity'
    elif ax_y == 'Cloudiness':
        y_label= 'Cloudiness (%)'
        plot_title = 'City Latitude vs. Cloudiness'
    elif ax_y == 'Wind Speed':
        y_label= 'Wind Speed (mph)'
        plot_title = 'City Latitude vs. Cloudiness'
    else:
        y_label= 'N/A'
        plot_title = 'N/A'
   
    today=now.strftime("%m/%d/%Y")
    plot_title=plot_title+ ' ('+today+')'
    # Incorporate the other graph properties
    plt.title(plot_title)
    plt.ylabel(y_label)
    plt.xlabel(x_label)
    plt.grid(True)
    
    # Save Figure
    fig_title = (f'{plot_title}.png')
    plt.savefig('pyber ride sharing data.png')
    
    # Show plot
    plt.show()

In [ ]:
generic_plot('Lat','Max Temp')

#### Latitude vs. Humidity Plot

In [ ]:
generic_plot('Lat','Humidity')

#### Latitude vs. Cloudiness Plot

In [ ]:
generic_plot('Lat','Cloudiness')

#### Latitude vs. Wind Speed Plot

In [ ]:
generic_plot('Lat','Wind Speed')